In [42]:
#0.Params
Rho_window = 120
resid_ma_window = 120
vwap_window_short = 21
twap_window_short = 21
vwap_window_medium = 42
twap_window_medium = 42

#1.Import libraries
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

#2.Load File
ALT_csv = 'Market_Bybit_ASTERUSDT_1min_20250901-20250926'
df_raw = pd.read_csv(f'01.data/{ALT_csv}.csv')

#3.Calculate Foundational Numbers
df = df_raw.copy()
df['Typical_price'] = (df['O'] + df['H'] + df['L'] + df['C']) / 4
df['TWAP_short']= df['Typical_price'].rolling(twap_window_short, min_periods=1).mean()
df['TWAP_medium']= df['Typical_price'].rolling(twap_window_medium, min_periods=1).mean()

df['TPxV'] = df['Typical_price'] * df['V']
roll_tpv_short = df["TPxV"].rolling(vwap_window_short, min_periods=1).sum()
roll_v_short   = df['V'].rolling(vwap_window_short, min_periods=1).sum()
df['VWAP_short'] = roll_tpv_short / roll_v_short
roll_tpv_medium = df["TPxV"].rolling(vwap_window_medium, min_periods=1).sum()
roll_v_medium   = df['V'].rolling(vwap_window_medium, min_periods=1).sum()
df['VWAP_medium'] = roll_tpv_medium / roll_v_medium

#3.Generate Features
df['TP_logret'] = np.log(df['Typical_price']/df['Typical_price'].shift(1))
df['TP_logret_lag1'] = df['TP_logret'].shift(1)
df['Rho_TP_logret_roll(1)'] = df['TP_logret'].rolling(Rho_window, min_periods=Rho_window//3).corr(df['TP_logret'].shift(1))

df['TWAP_logret'] = np.log(df['TWAP_short']/df['TWAP_short'].shift(1))
df['TWAP_logret_lag1'] = df['TWAP_logret'].shift(1)
df['Rho_TWAP_logret_roll(1)'] = df['TWAP_logret'].rolling(Rho_window, min_periods=Rho_window//3).corr(df['TWAP_logret'].shift(1))

phi = df['TP_logret'].autocorr(lag=1)   # AR(1)係数の推定
df['resid'] = df['TP_logret'] - phi * df['TP_logret'].shift(1)
df['resid_abs'] = df['resid'].abs()
df['resid_abs_ma'] = df['resid_abs'].rolling(resid_ma_window, min_periods =10).mean()
df['resid_abs_stdev'] = df['resid'].rolling(resid_ma_window).std()


In [43]:
#99.Create Graphs

#Slicing
df_graph = df.copy()
# timestamp を datetime 型にしておく（既にそうなら不要）
df_graph["timestamp"] = pd.to_datetime(df_graph["timestamp"])
# 任意の期間を指定
start = "2025-09-23"
end   = "2025-09-24"
df_graph = df_graph[(df_graph["timestamp"] >= start) & (df_graph["timestamp"] <= end)]

#Graph body
fig = make_subplots(
    rows = 4, cols = 1,
    shared_xaxes = True,
    vertical_spacing = 0.02
)

fig.add_trace(
    go.Candlestick(
        x = df_graph['timestamp'], open = df_graph['O'], high = df_graph['H'], low = df_graph['L'], close = df_graph['C'],
        name = 'OHLC', showlegend = False),
    row = 1, col = 1
)

fig.add_trace(
    go.Scatter(x = df_graph['timestamp'], y = df_graph['TWAP_short'], mode = 'lines', name = 'TWAP_short'),
    row = 2, col = 1
)

fig.add_trace(
    go.Scatter(
        x = df_graph['timestamp'], y = df_graph['TWAP_medium'], mode = 'lines', name = 'TWAP_medium'),
        row = 2, col = 1
)

fig.add_trace(
    go.Scatter(
        x = df_graph['timestamp'], y = df_graph['Rho_TWAP_logret_roll(1)'], mode = 'lines', name = 'Rho_TP_logret_roll(1)'),
        row = 3, col = 1
)

fig.add_trace(
    go.Scatter(
        x = df_graph['timestamp'], y = df_graph['resid_abs_stdev'], mode = 'lines', name = 'resid_abs'),
        row = 4, col = 1
)

fig.update_xaxes(
    rangeslider=dict(visible=False),
    row=1, col=1
)

fig.update_layout(
    height=900,   # デフォルトはだいたい 450。数値を大きくすると縦長になる
    hovermode='x unified',
    title='TP/TWAP と Rolling Autocorr (lag=1)',
    margin=dict(t=60, l=40, r=20, b=40),
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1.0)
)


fig.show()